In [21]:
from typing import TypedDict, Dict 
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.runnables.graph import MermaidDrawMethod
from langchain_groq import ChatGroq
from IPython.display import display, Image  
from dotenv import load_dotenv
import os
import gradio as gr

In [29]:
#loading the api keys
load_dotenv() 
groq_api_key = os.getenv("GROQ_API_KEY")

# Check if the key is loaded successfully
if groq_api_key:
    print("Groq API Key loaded successfully!")
else:
    print("Error: Groq API Key not found. Check your .env file.")



Groq API Key loaded successfully!


TypeDict is a dictionary that allows for definition of the expected structure of dictionary objects including the type of each key-value pair while still using the familiar dictionary syntax.
A class named State is defined using TypeDict and it has four attributes;
query: str - holds a string that represents a customer query or question.
category: str - store a string that categorizes the query. In this particular case, technical,billing and general.
sentiment: str - captures the sentiment of the query
response: str - hold the response generated for the query 

In [3]:
#Creating the state class
class State(TypedDict): 
    query: str
    category: str
    sentiment: str 
    response: str 

example_state: State = {
    "query": "What are transducers?",
    "category": "technical",
    "sentiment": "neutral",
    "response": "Transducers are devices that convert one form of energy into another."
}

print(f"Query: {example_state['query']}")
print(f"Category: {example_state['category']}")
print(f"Sentiment: {example_state['sentiment']}")
print(f"Response: {example_state['response']}")
    

Query: What are transducers?
Category: technical
Sentiment: neutral
Response: Transducers are devices that convert one form of energy into another.


ChatGroq() creates a model using Groq
temperature = 0, - the model will only generate straightforward & focused responses
groq_api_key = groq_api_key - the password to use the model
model_name = "llama-3.3-70b-versatile" - the specific llm is llama

In [4]:
#Defining the llm
llm = ChatGroq( 
    temperature = 0,
    groq_api_key = groq_api_key,
    model_name = "llama-3.3-70b-versatile"
)

.invoke() sends a prompt (the string "what is a cat") to the AI and gets a response
.content extracts the text answer from the result object

In [5]:
#testing the model
result = llm.invoke("what is the capital city of australia")
result.content

'The capital city of Australia is Canberra.'

**def categorize(state : State) -> State:**
The function categorize takes an input parameter called state, State (the dictionary above), -> State makes sure the fucntion will return something of the same State type

**prompt = ChatPromptTemplate.from_template** - 
Creates a template to for structuring queries into the three categories

**chain = prompt | llm**- 
| (pipe) operator connects the prompt template and the language model and links the prompt to the llm for processing

**category = chain.invoke({"query" : state['query']}).content** - 
Gets query from the state dicionary, sends it to the chain and triggers the model to process and respond to the query.
.content gives back the text output

**return {"category": category}**- this is a function that returs the dictionary "category" associated with the value of the variable category

In [6]:


#categorizing function
def categorize(state : State) -> State:
     "Technical, Billing, General"
     prompt = ChatPromptTemplate.from_template(
      "Categorize the following customer query into one of these categories: "
      "Technical, Billing, General. Query: {query}"
     )
     chain = prompt | llm
     category = chain.invoke({"query" : state['query']}).content
     return {"category": category}


**def analyze_sentiment(state: State) -> State:**
defines a function that takes a state object as input and returns a new state object

**prompt = ChatPromptTemplate.from_template(
    "Analyze the sentiment of the following customer query. "
    "Respond with either 'Positive', 'Neutral', or 'Negative'. Query: {query}"
)** - This creates a prompt template and includes instructions to analyze the sentiment of a customer query and respond with one of three possible sentiments: 'Positive', 'Neutral', or 'Negative'. The {query} will be replaced with the actual query from the input state.

**chain = prompt | llm**- 
| (pipe) operator connects the prompt template and the language model and links the prompt to the llm for processing

**sentiment = chain.invoke({"query": state['query']}).content** - a chain is created then executed with the query from the state. The response is stored in the sentiment variable.


In [7]:
# analyzing sentiment function
def analyze_sentiment(state: State) -> State:
    prompt = ChatPromptTemplate.from_template(
        "Analyze the sentiment of the following customer query. "
        "Respond with either 'Positive', 'Neutral', or 'Negative'. Query: {query}"
    )
    chain = prompt | llm
    sentiment = chain.invoke({"query": state['query']}).content  # Capture sentiment
    return {"sentiment": sentiment}  # Return sentiment instead of category

**def handle_technical(state : State) -> State:** - 
The function handle_technical takes a state object as input and returns a new state object.

**prompt = ChatPromptTemplate.from_template("Provide a technical support response to the following query : {query}")** - 
Instructs the model to provide support response to the query

**chain = prompt | llm**- 
| (pipe) operator connects the prompt template and the language model and links the prompt to the llm for processing

**response = chain.invoke({"query" : state['query']}).content return {"response" : response}**
executes the chain from the input and returns an output response 

In [8]:

#handle technical function
def handle_technical(state : State) -> State:
    prompt = ChatPromptTemplate.from_template(
        "Provide a technical support response to the following query : {query}"
        )
    chain = prompt | llm
    response = chain.invoke({"query" : state['query']}).content
    return {"response" : response}

Creates a prompt template for a general support response and pipes the prompt into the llm to create a chain and invokes it with the query from the state object. The response is captured from the chain invocation.

In [9]:
#handling billing function
def handle_billing(state : State) -> State:
    prompt = ChatPromptTemplate.from_template(
        "Provide a billing support response to the following query : {query}"

    )
    chain = prompt | llm
    response = chain.invoke({"query" : state['query']}).content
    return {"response" : response}

#handle general
def handle_general(state : State) -> State:
    prompt = ChatPromptTemplate.from_template(
        "Provide a general support response to the following query : {query}"

    )
    chain = prompt | llm
    response = chain.invoke({"query" : state['query']}).content
    return {"response" : response}

#escalate the situation
def escalate(state : State) -> State:
    return{"response" : "This query has been escalate to a human agent due to its negative sentiment"}

If the sentiment of the query is negative, it returns the string "escalate".
If the sentiment is not negative, it checks the category of the query.
If the category is "Technical", it returns the string "handle_technical".
If the category is "Billing", it returns the string "handle_billing".
If the category doesn't match any of the above conditions, it returns the string "handle_general"

In [10]:
# handling the routing function
def route_query(state: State) -> State:
    if state['sentiment'] == 'Negative':
        return "escalate"
    elif state['category'] == 'Technical':
        return "handle_technical"
    elif state['category'] == 'Billing':
        return "handle_billing"
    else:
        return "handle_general"

### Query Handling Workflow

* Start by **categorizing** the query.  
* Check the **sentiment** of the query.  
* Send the query to the right handler based on category and sentiment:  
   - Technical → `handle_technical`  
   - Billing → `handle_billing`  
   - General → `handle_general`  
   - Negative sentiment → `escalate`  
* Each handler finishes the process.  
* The workflow is compiled into `app`.

In [13]:
#crafting the workflow
workflow = StateGraph(State)
workflow.add_node("categorize", categorize)
workflow.add_node("analyze_sentiment", analyze_sentiment)
workflow.add_node("handle_technical", handle_technical)
workflow.add_node("handle_billing", handle_billing)
workflow.add_node("handle_general", handle_general)
workflow.add_node("escalate", escalate)

#adding edges
workflow.add_edge("categorize", "analyze_sentiment")
workflow.add_conditional_edges("analyze_sentiment", 
route_query,{
    "handle_technical" : "handle_technical",
    "handle_billing": "handle_billing",
    "handle_general" : "handle_general",
    "escalate": "escalate"
})
workflow.add_edge("handle_technical", END)
workflow.add_edge("handle_billing", END)
workflow.add_edge("handle_general", END)
workflow.add_edge("escalate", END)

#aadding the entry point
workflow.set_entry_point("categorize")

#compile the workflow
app = workflow.compile()

In [33]:
#visualizing the workflow
display(
    Image(
        app.get_graph().draw_mermaid_png(
            draw_method = MermaidDrawMethod.API
        )

))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [ ]:
Function to run customer support
def run_customer_support(query: str) -> dict:
    result = app.invoke({"query": query})  
    return {
        "category": result['category'], 
        "sentiment": result['sentiment'],
        "response": result['response'],
    }

In [26]:
#testing the output
query = "I need to bake a cake. Can you help me?"
result = run_customer_support(query)
print(f"Query: {query}")
print(f"Category: {result['category']}")
print(f"Sentiment: {result['sentiment']}")
print(f"Response: {result['response']}")
print("\n")

Query: I need to bake a cake. Can you help me?
Category: I would categorize the customer query as "General". The query is not related to a specific product or service issue (Technical) and does not involve payment or account information (Billing). It appears to be a general inquiry or request for assistance with a non-technical topic, in this case, baking a cake.
Sentiment: Neutral. The customer is simply stating a need and asking for help, without expressing any emotion or opinion that would indicate a positive or negative sentiment.
Response: Baking a cake can be a fun and rewarding experience. I'd be happy to help you with that. To get started, could you please provide me with a bit more information about the cake you want to bake? For example:

* What type of cake are you looking to make (e.g. vanilla, chocolate, birthday cake, etc.)?
* How many people are you planning to serve?
* Do you have any specific ingredients or dietary restrictions in mind (e.g. gluten-free, vegan, etc.)?


#### Building a Simple UI for a Customer Support Assistant
* Developing a  User Interface where you will enter your query, and the system will:  

- Determine the **Category** (Technical, Billing, or General).  
-  Analyze the **Sentiment** (Positive, Neutral, or Negative).  
- Provide an appropriate **Response** or escalate if needed.  

In [20]:
def gradio_interface(query: str):
    result = run_customer_support(query)  
    return (
        f"**Category:** {result['category']}\n\n"
        f"**Sentiment:** {result['sentiment']}\n\n"
        f"**Response:** {result['response']}"
    )

# Building the Gradio app
gui = gr.Interface(
    fn=gradio_interface,
    theme='Yntec/HaleyCH_Theme_Orange_Green',
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs=gr.Markdown(),
    title="Customer Support Assistant",
    description="Ask a question and get a categorized, sentiment-aware response from the system."
)

# Launching the app
if __name__ == "__main__":
    gui.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
